TCN Model to predict the need for Mechnical Ventilation
Signals Used: PPG, ECG I,II,III,V

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.axis import Axis
from sklearn.metrics import mean_squared_error
import keras

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from datetime import datetime
from packaging import version
import glob
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.axis import Axis
from sklearn.metrics import mean_squared_error
import keras

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import InputLayer
from tensorflow.keras import Input, Model
from tcn import TCN, tcn_full_summary
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
print("TF version:" , tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [ ]:
print(tf.version.VERSION)
print(keras.__version__)

In [ ]:
print(physical_devices)

In [ ]:
num_signal = 5
time = 6*60
period = 60
slots_permin = 1
num_half_data_chunks = 6*60
num_points_perdatapoint = period * 240 
num_total_patients = 87
num_training_patients = 60
num_validation_patients = 10
num_testing_patients = 17
num_half_data_points = num_total_patients*240*60*time


In [ ]:
#np.save("hours6_allsignal_admit",Data0)

In [ ]:
Data = np.load("hours6_allsignal_complete.npy")

In [ ]:
print(Data)

In [ ]:
#Uncomment to visualize the dataset
'''
plt.rcParams["figure.figsize"] = (20,150)
figure, axis = plt.subplots(27, 3)
for i in range(3):
    for j in range(27):
        axis[j, i].plot(Data0[j*3+i])
        axis[j, i].set_title("Patient # {}" .format(j*3+i))
'''

In [ ]:
'''
plt.rcParams["figure.figsize"] = (20,150)
figure, axis = plt.subplots(27, 3)
for i in range(3):
    for j in range(27):
        axis[j, i].plot(Data1[j*3+i])
        axis[j, i].set_title("Patient # {}" .format(j*3+i))
'''

In [ ]:
Data = np.nan_to_num(Data)

In [ ]:
print(Data.shape)
print(Data.shape[0]/240/60/60,"Hours")

In [ ]:
print(Data.shape)

In [ ]:
#Setting up training,validation and test set while isolating patients in each set.
Training_data = np.zeros((2*num_training_patients*time*60*240,num_signal))
validation_data = np.zeros((2*num_validation_patients*time*60*240,num_signal))
Testing_data = np.zeros((2*num_testing_patients*time*60*240,num_signal))

offset = time*60*240
Training_data[:num_training_patients*offset,:]  = Data[17*offset:77*offset,:]
validation_data[:num_validation_patients*offset,:] = Data[77*offset:87*offset,:]
Testing_data[:num_testing_patients*offset,:] = Data[0*offset:17*offset,:]

Training_data[num_training_patients*offset:num_training_patients*2*offset,:] = Data[(87+17)*offset:(87+17+60)*offset,:]
validation_data[num_validation_patients*offset:num_validation_patients*2*offset,:] = Data[(87+17+60)*offset:(87+17+60+10)*offset,:]
Testing_data[num_testing_patients*offset:num_testing_patients*2*offset,:] = Data[(87)*offset,:(87+17)*offset]

print(Training_data.shape)
print(validation_data.shape)
print(Testing_data.shape)

In [ ]:
Training_data = Training_data.reshape(num_training_patients*2*num_half_data_chunks,period*240,num_signal)
validation_data = validation_data.reshape(num_validation_patients*2*num_half_data_chunks,period*240,num_signal)
Testing_data = Testing_data.reshape(num_testing_patients*2*num_half_data_chunks,period*240,num_signal)
print(Training_data.shape)
print(validation_data.shape)
print(Testing_data.shape)

In [ ]:
Label0 = np.zeros((num_training_patients,num_half_data_chunks))
Label1 = np.ones((num_training_patients,num_half_data_chunks))
Training_Label = np.concatenate((Label0, Label1), axis=0)
print(Training_Label)
print(Training_Label.shape)

In [ ]:
Label0 = np.zeros((num_validation_patients,num_half_data_chunks))
Label1 = np.ones((num_validation_patients,num_half_data_chunks))
validation_Label = np.concatenate((Label0, Label1), axis=0)
print(validation_Label.shape)

In [ ]:
Label0 = np.zeros((num_testing_patients,num_half_data_chunks))
Label1 = np.ones((num_testing_patients,num_half_data_chunks))
Testing_Label = np.concatenate((Label0, Label1), axis=0)
print(Testing_Label.shape)

In [ ]:
Training_Label= Training_Label.reshape(num_training_patients*2*num_half_data_chunks)
validation_Label= validation_Label.reshape(num_validation_patients*2*num_half_data_chunks)
Testing_Label= Testing_Label.reshape(num_testing_patients*2*num_half_data_chunks)
print(Training_Label.shape)
print(validation_Label.shape)
print(Testing_Label.shape)

In [ ]:
'''
plt.rcParams["figure.figsize"] = (20,150)
figure, axis = plt.subplots(22, 3)
for i in range(3):
    for j in range(22):
        axis[j, i].plot(Training_data[j*3+i])
        axis[j, i].set_title("Patient # {}" .format(j*3+i))
'''

In [1]:
'''
figure, axis = plt.subplots(6, 3)
plt.rcParams["figure.figsize"] = (5,10)

for i in range(3):
    for j in range(6):
        axis[j, i].plot(Testing_data[j*3+i])
        axis[j, i].set_title("Patient # {}" .format(67+j*3+i))
'''

'\nfigure, axis = plt.subplots(6, 3)\nplt.rcParams["figure.figsize"] = (5,10)\n\nfor i in range(3):\n    for j in range(6):\n        axis[j, i].plot(Testing_data[j*3+i])\n        axis[j, i].set_title("Patient # {}" .format(67+j*3+i))\n'

In [ ]:
print(Testing_Label)

In [ ]:
from sklearn.utils import shuffle
Training_data, Training_Label = shuffle(Training_data,Training_Label,random_state = 2021)
validation_data, validation_Label = shuffle(validation_data, validation_Label,random_state = 2021)
Testing_data, Testing_Label = shuffle(Testing_data, Testing_Label,random_state = 2021)

In [ ]:
#Start of TCN model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import InputLayer
from tensorflow.keras import Input, Model
from tcn import TCN, tcn_full_summary
from sklearn.metrics import mean_squared_error

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
#TCN Model Architecture
input_dim = 240*60
output_dim = 1
batch_size, timesteps, input_dim = None, 14400, num_signal

tcn_layer = TCN(input_shape=(14400, num_signal),padding='same',nb_stacks = 1,return_sequences=False, nb_filters=64,kernel_size=5, dropout_rate=0.3,use_batch_norm=True)
# The receptive field tells you how far the model can see in terms of timesteps.
print('Receptive field size =', tcn_layer.receptive_field)

m = Sequential([
    tcn_layer,
    Dense(output_dim)
])
m.compile(optimizer='adam', loss='mse',metrics=[tf.keras.metrics.BinaryAccuracy()])


In [ ]:
'''
class TestCallback(keras.callbacks.Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))
'''

In [ ]:
m.summary()

In [ ]:
m.load_weights('./checkpoints/hours6_allsignal_model10')

In [ ]:
Data = []

In [ ]:
history = m.fit(x=Training_data, y=Training_Label,batch_size=64, epochs=10, validation_split=0.2, verbose =1,validation_data = (validation_data,validation_Label))

In [ ]:
m.save_weights('./checkpoints/hours6_allsignal_model10')

In [ ]:
print("Evaluate on test data")
results = m.evaluate(Testing_data, Testing_Label)
print("test loss, test acc:", results)


In [ ]:
predict = m.predict(Testing_data)
print(predict[0:20])
print(predict.max())
print(predict.min())
print(Testing_Label[0:20])

In [ ]:
#Confusion Matrix
'''
def predict_class(n):
    min_p = 0
    max_p = 1
    if (((n-min_p)/max_p)>0.5):
        return 1
    else:
        return 0

pred = np.array(list(map(predict_class,predict)))
print(pred)

from sklearn.metrics import confusion_matrix
from pandas import DataFrame
import seaborn as sn

columns = ["Do not need MV","Need MV"]
confm = confusion_matrix(Testing_Label, list(pred))
df_cm = DataFrame(confm, index=columns, columns=columns)
ax = sn.heatmap(df_cm, cmap='Blues', annot=True)
'''

In [ ]:
print(pred.shape)
print(Testing_Label.shape)

In [ ]:
plt.plot(history.history['binary_accuracy'])
plt.xlabel("Last Iterations")
plt.ylabel("Binary_Accuracy")

In [ ]:
plt.plot(history.history['loss'])
plt.xlabel("Last Iterations")
plt.ylabel("Loss")